In [1]:
%cd C:\Users\yashs\source\repos\Ripa-Shah\Sentiment-Analysis

C:\Users\yashs\source\repos\Ripa-Shah\Sentiment-Analysis


In [4]:
pip install tweepy praw requests pandas textblob vaderSentiment


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import praw

reddit = praw.Reddit(
    client_id="M9iSExe3kRkwrILVO75-CA",       # <-- from the app page
    client_secret="rnuYodBu7llgTAmssHujZVUt37mrAA",  # <-- from the app page
    user_agent="crypto-sentiment-app by u/MindlessEssay2919"  # <-- your Reddit username here
)

subreddit = reddit.subreddit("CryptoCurrency")

for post in subreddit.hot(limit=10):
    print(post.title)


Pineapple x MotionDEV: decentralizing healthcare | win $250 worth of MDEV
Announcing the MON public sale: the first ever token sale on Coinbase
Crypto millionaire dies in supercar crash after being blacklisted by banks
Crime Season Doubters, in Shambles.
The Trump Family Now Holds $1.2 Billion Worth of Bitcoins - a2z Cryptocurrencies
Many Indians working abroad send money home with USDT trades at a 4-5% premium, making crypto remittances more valuable than bank transfers.
House Rep. Demands Answers on Why the President Pardoned Billionaire Crypto Mogul
SoFi becomes first U.S. national bank to offer crypto trading
All It Takes Is A Tweet
China Accuses U.S. of Stealing 13 Billion USD In Crypto in High-Profile Crypto Hack


In [5]:
import praw
import pandas as pd
from datetime import datetime

# ✅ Reddit API connection
reddit = praw.Reddit(
    client_id="M9iSExe3kRkwrILVO75-CA",
    client_secret="rnuYodBu7llgTAmssHujZVUt37mrAA",
    user_agent="crypto-sentiment-app by u/MindlessEssay2919"
)

# ✅ Subreddit to scrape
subreddit = reddit.subreddit("CryptoCurrency")

# ✅ Define which cryptocurrency names you want to detect
crypto_keywords = ["bitcoin", "btc", "ethereum", "eth", "cardano", "ada", "dogecoin", "doge", "solana", "sol"]

# ✅ Create an empty list to store posts
data = []

# ✅ Extract top 50 hot posts
for post in subreddit.hot(limit=50):
    # Skip stickied posts
    if post.stickied:
        continue

    # Extract data
    date = datetime.utcfromtimestamp(post.created_utc).strftime("%Y-%m-%d %H:%M:%S")
    user = str(post.author)
    title = post.title
    review = post.selftext

    # Detect cryptocurrency mentioned in title or review
    text_combined = f"{title} {review}".lower()
    crypto_found = [c for c in crypto_keywords if c in text_combined]
    cryptocurrency = ", ".join(set(crypto_found)) if crypto_found else "unknown"

    # Append record
    data.append([date, user, title, cryptocurrency, review])

# ✅ Convert to DataFrame
df = pd.DataFrame(data, columns=["Date", "UserID", "Title", "CryptocurrencyName", "Review"])

# ✅ Save to CSV
df.to_csv("reddit_crypto_posts.csv", index=False, encoding="utf-8-sig")

print("✅ Data successfully saved to 'reddit_crypto_posts.csv'")
print(df.head())


✅ Data successfully saved to 'reddit_crypto_posts.csv'
                  Date              UserID  \
0  2025-11-11 12:35:39        TheExpressUS   
1  2025-11-11 14:46:13            GabeSter   
2  2025-11-11 15:40:20          emperordas   
3  2025-11-11 01:50:32  According_Time5120   
4  2025-11-11 06:28:21       Green_Candler   

                                               Title CryptocurrencyName  \
0  Crypto millionaire dies in supercar crash afte...            unknown   
1                Crime Season Doubters, in Shambles.            unknown   
2  The Trump Family Now Holds $1.2 Billion Worth ...            bitcoin   
3  Many Indians working abroad send money home wi...            unknown   
4  House Rep. Demands Answers on Why the Presiden...            unknown   

                                              Review  
0                                                     
1  price screenshot from yesterday (all 3 were to...  
2  Trump, who won as a pro-Bitcoin candidate, hol...

C:\Users\yashs\AppData\Local\Temp\ipykernel_15276\3055511292.py:28: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(post.created_utc).strftime("%Y-%m-%d %H:%M:%S")


In [ ]:
import praw
import pandas as pd
import time
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize Reddit API
reddit = praw.Reddit(
    client_id="M9iSExe3kRkwrILVO75-CA",
    client_secret="rnuYodBu7llgTAmssHujZVUt37mrAA",
    user_agent="crypto-sentiment-app by u/MindlessEssay2919"
)

# Define cryptocurrency keywords to track
crypto_keywords = {
    "bitcoin": "BTC",
    "btc": "BTC",
    "ethereum": "ETH",
    "eth": "ETH",
    "cardano": "ADA",
    "ada": "ADA",
    "dogecoin": "DOGE",
    "doge": "DOGE",
    "solana": "SOL",
    "sol": "SOL"
}

# CSV file path
CSV_FILE = "reddit_crypto_sentiment.csv"

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize CSV file (only once)
df_init = pd.DataFrame(columns=[
    "date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"
])
df_init.to_csv(CSV_FILE, index=False)

print("🔄 Starting Reddit post & comment sentiment streaming...")
print("Press Ctrl+C to stop.\n")

try:
    while True:
        records = []

        # Fetch latest posts
        for post in reddit.subreddit("CryptoCurrency").new(limit=10):
            title = post.title
            review = post.selftext if post.selftext else ""
            user_id = str(post.author) if post.author else "Anonymous"
            date = datetime.utcfromtimestamp(post.created_utc).strftime("%Y-%m-%d %H:%M:%S")

            # Detect which crypto is mentioned
            text_lower = (title + " " + review).lower()
            found_coin = None
            for keyword, symbol in crypto_keywords.items():
                if keyword in text_lower:
                    found_coin = symbol
                    break
            if not found_coin:
                continue

            # Sentiment for post
            sentiment = analyzer.polarity_scores(title + " " + review)["compound"]
            records.append([date, user_id, "post", title, found_coin, review, sentiment])
            print(f"{date} | {user_id} | POST | {found_coin} | Sentiment: {sentiment:+.3f}")

            # Fetch top-level comments
            post.comments.replace_more(limit=0)
            for comment in post.comments.list()[:10]:  # limit comments per post
                comment_text = comment.body
                comment_user = str(comment.author) if comment.author else "Anonymous"
                comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")

                # Skip deleted or removed comments
                if not comment_text or comment_text in ["[deleted]", "[removed]"]:
                    continue

                # Detect crypto mention in comment
                comment_lower = comment_text.lower()
                comment_coin = found_coin
                for keyword, symbol in crypto_keywords.items():
                    if keyword in comment_lower:
                        comment_coin = symbol
                        break

                # Sentiment for comment
                comment_sent = analyzer.polarity_scores(comment_text)["compound"]
                records.append([comment_date, comment_user, "comment", title, comment_coin, comment_text, comment_sent])

        # Save to CSV
        if records:
            df = pd.DataFrame(records, columns=[
                "date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"
            ])
            df.to_csv(CSV_FILE, mode="a", header=False, index=False)
            print(f"✅ Saved {len(records)} new entries to {CSV_FILE}")

        # Wait before next update
        time.sleep(120)  # every 2 minutes

except KeyboardInterrupt:
    print("\n🛑 Stopped streaming by user.")


🔄 Starting Reddit post & comment sentiment streaming...
Press Ctrl+C to stop.



C:\Users\yashs\AppData\Local\Temp\ipykernel_15276\543877309.py:52: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(post.created_utc).strftime("%Y-%m-%d %H:%M:%S")


2025-11-11 17:16:51 | Apart_Trouble476 | POST | BTC | Sentiment: -0.487


C:\Users\yashs\AppData\Local\Temp\ipykernel_15276\543877309.py:74: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")


2025-11-11 16:58:43 | KIG45 | POST | BTC | Sentiment: +0.000
2025-11-11 16:20:59 | dailymail | POST | BTC | Sentiment: -0.477
2025-11-11 16:04:21 | Defiant-Branch4346 | POST | BTC | Sentiment: +0.000
2025-11-11 15:40:20 | emperordas | POST | BTC | Sentiment: +0.872
2025-11-11 15:15:18 | KazuFromUniswap | POST | BTC | Sentiment: +0.000
2025-11-11 15:04:40 | yahoofinance | POST | BTC | Sentiment: +0.854
✅ Saved 32 new entries to reddit_crypto_sentiment.csv


In [ ]:
pip install yfinance pandas

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ------------- -------------------------- 1.0/3.0 MB 7.4 MB/s eta 0:00:01
     -------------------------------------- - 2.9/3.0 MB 7.8 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 7.6 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pypro

In [1]:
import yfinance as yf

# Create a Ticker object for Bitcoin in USD
btc = yf.Ticker("BTC-USD")

# Get the current price
btc_price = btc.history(period="1d", interval="1m")  # 1-minute intervals for near real-time
print(btc_price.tail())


                                    Open           High            Low  \
Datetime                                                                 
2025-11-11 17:42:00+00:00  102958.500000  102958.500000  102958.500000   
2025-11-11 17:43:00+00:00  102957.492188  102957.492188  102957.492188   
2025-11-11 17:44:00+00:00  102920.382812  102920.382812  102920.382812   
2025-11-11 17:45:00+00:00  102962.585938  102962.585938  102962.585938   
2025-11-11 17:46:00+00:00  102961.531250  102961.531250  102961.531250   

                                   Close     Volume  Dividends  Stock Splits  
Datetime                                                                      
2025-11-11 17:42:00+00:00  102958.500000          0        0.0           0.0  
2025-11-11 17:43:00+00:00  102957.492188  135585792        0.0           0.0  
2025-11-11 17:44:00+00:00  102920.382812   26083328        0.0           0.0  
2025-11-11 17:45:00+00:00  102962.585938   13352960        0.0           0.0  
2025-11

In [ ]:
import yfinance as yf
import pandas as pd
import time
from datetime import datetime

# Define the cryptocurrencies to track
TICKERS = ["BTC-USD", "ETH-USD", "ADA-USD", "DOGE-USD", "SOL-USD"]

# CSV file path
CSV_FILE = "crypto_prices.csv"

# Initialize CSV file with headers (only once)
df_init = pd.DataFrame(columns=["timestamp", "symbol", "price"])
df_init.to_csv(CSV_FILE, index=False)

print("Starting live crypto price streaming...\nPress Ctrl+C to stop.")

try:
    while True:
        # Get latest prices
        data = yf.download(tickers=TICKERS, period="1d", interval="1m")["Close"].tail(1)

        # Convert to long format
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        records = []

        for symbol in TICKERS:
            price = data[symbol].values[0]
            records.append([timestamp, symbol, price])
            print(f"{timestamp} | {symbol} | ${price:.4f}")

        # Append to CSV file
        df = pd.DataFrame(records, columns=["timestamp", "symbol", "price"])
        df.to_csv(CSV_FILE, mode="a", header=False, index=False)

        # Wait before next update (e.g., every 60 seconds)
        time.sleep(60)

except KeyboardInterrupt:
    print("\nStreaming stopped by user.")


Starting live crypto price streaming...
Press Ctrl+C to stop.


C:\Users\yashs\AppData\Local\Temp\ipykernel_13692\4025485919.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=TICKERS, period="1d", interval="1m")["Close"].tail(1)
[*********************100%***********************]  5 of 5 completed


2025-11-16 13:22:17 | BTC-USD | $93732.3438
2025-11-16 13:22:17 | ETH-USD | $3072.2866
2025-11-16 13:22:17 | ADA-USD | $0.4794
2025-11-16 13:22:17 | DOGE-USD | $0.1556
2025-11-16 13:22:17 | SOL-USD | $135.9897


C:\Users\yashs\AppData\Local\Temp\ipykernel_13692\4025485919.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=TICKERS, period="1d", interval="1m")["Close"].tail(1)
[*********************100%***********************]  5 of 5 completed


2025-11-16 13:23:17 | BTC-USD | $93757.5234
2025-11-16 13:23:17 | ETH-USD | $3073.4153
2025-11-16 13:23:17 | ADA-USD | $0.4795
2025-11-16 13:23:17 | DOGE-USD | $0.1556
2025-11-16 13:23:17 | SOL-USD | $135.9769


...

2025-11-16 13:34:24 | BTC-USD | $93586.7109
2025-11-16 13:34:24 | ETH-USD | $3062.2917
2025-11-16 13:34:24 | ADA-USD | $0.4777
2025-11-16 13:34:24 | DOGE-USD | $0.1552
2025-11-16 13:34:24 | SOL-USD | $135.5654


C:\Users\yashs\AppData\Local\Temp\ipykernel_13692\4025485919.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=TICKERS, period="1d", interval="1m")["Close"].tail(1)
[*********************100%***********************]  5 of 5 completed


2025-11-16 13:35:25 | BTC-USD | $93631.0156
2025-11-16 13:35:25 | ETH-USD | $3064.4485
2025-11-16 13:35:25 | ADA-USD | $0.4777
2025-11-16 13:35:25 | DOGE-USD | $0.1552
2025-11-16 13:35:25 | SOL-USD | $135.5720


C:\Users\yashs\AppData\Local\Temp\ipykernel_13692\4025485919.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=TICKERS, period="1d", interval="1m")["Close"].tail(1)
[*********************100%***********************]  5 of 5 completed


2025-11-16 13:36:25 | BTC-USD | $93700.5391
2025-11-16 13:36:25 | ETH-USD | $3072.3860
2025-11-16 13:36:25 | ADA-USD | $0.4779
2025-11-16 13:36:25 | DOGE-USD | $0.1553
2025-11-16 13:36:25 | SOL-USD | $135.6941
